# Tutorial 15: Energy Analysis

Analyzing energy conservation and energy landscapes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mechanics_dsl import PhysicsCompiler

compiler = PhysicsCompiler()

dsl_code = r"""
\system{pendulum_energy}
\defvar{theta}{Angle}{rad}
\parameter{m}{1.0}{kg}
\parameter{L}{1.0}{m}
\parameter{g}{9.81}{m/s^2}
\lagrangian{\frac{1}{2}*m*L^2*\dot{theta}^2 + m*g*L*\cos{theta}}
\initial{theta=2.5, theta_dot=0.0}
"""

result = compiler.compile_dsl(dsl_code)
sol = compiler.simulate(t_span=(0, 15), num_points=1500)

In [ ]:
t = sol['t']
theta, theta_dot = sol['y'][0], sol['y'][1]
m, L, g = 1.0, 1.0, 9.81

# Energies
KE = 0.5 * m * L**2 * theta_dot**2
PE = -m * g * L * np.cos(theta)
E_total = KE + PE

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Energy vs time
axes[0,0].plot(t, KE, 'r-', label='Kinetic')
axes[0,0].plot(t, PE, 'b-', label='Potential')
axes[0,0].plot(t, E_total, 'k--', label='Total')
axes[0,0].set_xlabel('Time (s)')
axes[0,0].set_ylabel('Energy (J)')
axes[0,0].set_title('Energy vs Time')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Phase space with energy contours
theta_range = np.linspace(-np.pi, np.pi, 100)
omega_range = np.linspace(-10, 10, 100)
TH, OM = np.meshgrid(theta_range, omega_range)
E_landscape = 0.5 * m * L**2 * OM**2 - m * g * L * np.cos(TH)

axes[0,1].contour(TH, OM, E_landscape, levels=20, cmap='coolwarm', alpha=0.7)
axes[0,1].plot(theta, theta_dot, 'g-', lw=2, label='Trajectory')
axes[0,1].set_xlabel('θ (rad)')
axes[0,1].set_ylabel('θ̇ (rad/s)')
axes[0,1].set_title('Phase Space with Energy Contours')
axes[0,1].grid(True, alpha=0.3)

# Potential energy landscape
axes[1,0].plot(theta_range, -m*g*L*np.cos(theta_range), 'b-', lw=2)
axes[1,0].axhline(y=E_total[0], color='r', linestyle='--', label=f'E = {E_total[0]:.2f} J')
axes[1,0].set_xlabel('θ (rad)')
axes[1,0].set_ylabel('V(θ) (J)')
axes[1,0].set_title('Potential Energy Landscape')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Energy drift
axes[1,1].plot(t, (E_total - E_total[0])/E_total[0] * 100, 'purple', lw=1)
axes[1,1].set_xlabel('Time (s)')
axes[1,1].set_ylabel('ΔE/E₀ (%)')
axes[1,1].set_title('Energy Conservation Error')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()